In [1]:
from utils import load_rumors_from_jsonl
import os

out_dir = './temp-data'

clef_path = '../clef2024-checkthat-lab/task5'
data_path = os.path.join(clef_path, 'data')

filepath_train = os.path.join(data_path, 'English_train.json')
filepath_dev = os.path.join(data_path, 'English_dev.json')

train_jsons = load_rumors_from_jsonl(filepath_train)
dev_jsons = load_rumors_from_jsonl(filepath_dev)

print(f'loaded {len(train_jsons)} training json objects and {len(dev_jsons)} dev objects')

loaded 96 training json objects and 32 dev objects


In [2]:
import json
data_grouped_by_id_dev = {}
for entry in dev_jsons:
    data_grouped_by_id_dev[entry['id']] = entry
# print(json.dumps(data_grouped_by_id['AuRED_142'], indent=2))

In [3]:
import json
data_grouped_by_id_train = {}
for entry in train_jsons:
    data_grouped_by_id_train[entry['id']] = entry
# print(json.dumps(data_grouped_by_id['AuRED_142'], indent=2))

In [4]:
def trec_with_data(filename, data_grouped_by_id):
    with open(filename, 'r') as file:
        prev_rumor = ''
        ev_from_trec = []
        rumor_dict = None
        for line in file:
            try:
                rumor_id,q0,evidence_id,rank,score,label = line.split('\t')
                
                if rumor_dict and prev_rumor != rumor_id:
                    ev_not_found = [e[2] for e in rumor_dict['evidence'] if e[1] not in ev_from_trec]
                    if ev_not_found:
                        print('evidence not found:')
                        for text in ev_not_found:
                            print(text)
                    else:
                        print('all evidence found!')
                    ev_from_trec = []
                    print('\n')


                rumor_dict = data_grouped_by_id[rumor_id]
                if int(rank) == 1:
                    print(rumor_dict['label'], ':', rumor_dict['rumor'])

                ev_author, ev_id, ev_text = '', '', ''
                ev_ids = [e[1] for e in rumor_dict['evidence']]

                for ev in rumor_dict['timeline']:
                    if evidence_id == ev[1]:
                        ev_author, ev_id, ev_text = ev
                        ev_from_trec += [evidence_id]
                    
                print('(*)\t' if evidence_id in ev_ids else '\t',
                    rumor_id,'\t',
                    evidence_id,'\t',
                    rank,'\t',
                    f'{" " if float(score) > 0 else ""}{float(score):.4f}','\t',
                    ev_text,'\t',
                    f'{ev_author}/status/{ev_id}','\t',
                    )
                prev_rumor = rumor_id
            except KeyError:
                pass

In [5]:
# match up the retrieval scores from the dataset submission sample with the text from the tweets

tfidf_retrieve = 'temp-data/lucene-trec-dev.txt'

trec_with_data(tfidf_retrieve, data_grouped_by_id_dev)

REFUTES : Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD
(*)	 AuRED_142 	 1555986659279360001 	 1 	  3.9675 	 “The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4,000 people, and recalls that it had denied yesterday the false information published by the French magazine ‘Liberation’ about the same fabricated news. " 	 https://twitter.com/LBpresidency/status/1555986659279360001 	
(*)	 AuRED_142 	 1555424541509386240 	 2 	  2.2340 	 “The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news. 	 https://twitter.com/LBpresidency/status/1555424541509386240 	
	 AuRED_142 	 1555495801962614786 	 3 	  0.7894 	 President Aoun received the Lebanese Ambassador to Canada, Ambassador Fad

In [6]:
# match up the retrieval scores from the dataset submission sample with the text from the tweets

kgat_retrieve_sample = clef_path + '/submission_samples/KGAT_zeroShot_evidence_English_dev.txt'

trec_with_data(kgat_retrieve_sample, data_grouped_by_id_dev)

REFUTES : Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD
(*)	 AuRED_142 	 1555986659279360001 	 1 	 -0.7252 	 “The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4,000 people, and recalls that it had denied yesterday the false information published by the French magazine ‘Liberation’ about the same fabricated news. " 	 https://twitter.com/LBpresidency/status/1555986659279360001 	
(*)	 AuRED_142 	 1555424541509386240 	 2 	 -0.8598 	 “The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news. 	 https://twitter.com/LBpresidency/status/1555424541509386240 	
	 AuRED_142 	 1554433114746880000 	 3 	 -0.8770 	 President Aoun received the credentials of Iranian Ambassador Mojtaba Ama

In [7]:
# match up the retrieval scores from the dataset submission sample with the text from the tweets

tfidf_retrieve = 'temp-data/tfidf-trec.txt'

trec_with_data(tfidf_retrieve, data_grouped_by_id_dev)

REFUTES : Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD
(*)	 AuRED_142 	 1555424541509386240 	 1 	  0.1289 	 “The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news. 	 https://twitter.com/LBpresidency/status/1555424541509386240 	
	 AuRED_142 	 1555495801962614786 	 2 	  0.1236 	 President Aoun received the Lebanese Ambassador to Canada, Ambassador Fadi Ziadeh, who briefed him on the conditions of the Lebanese community in Canada, Lebanese-Canadian relations, and ways to develop them in all fields. https://t.co/HwgRPhBS5F 	 https://twitter.com/LBpresidency/status/1555495801962614786 	
	 AuRED_142 	 1555126458191855616 	 3 	  0.0982 	 President Aoun received the Ambassador of Finland, Tarja Fernandes, on a farewell visit on the occasion of the end of her diplomatic duties in Le